# VLM Benchmark for Object Property Abstraction

This notebook implements a benchmark for evaluating Vision Language Models (VLMs) on object property abstraction and visual question answering (VQA) tasks. The benchmark includes three types of questions:

1. Direct Recognition
2. Property Inference
3. Counterfactual Reasoning

And three types of images:
- REAL
- ANIMATED
- AI GENERATED

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [1]:
# Install required packages
!pip install transformers torch Pillow tqdm

In [2]:
# Import required libraries
import torch
import json
from pathlib import Path
from PIL import Image
import gc
import re
from tqdm import tqdm
import time
from typing import List, Dict, Any

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Benchmark Tester Class

This class handles the evaluation of models against our benchmark.

In [3]:
# class BenchmarkTester:
#     def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         with open(benchmark_path, 'r') as f:
#             self.benchmark = json.load(f)
#         self.data_dir = data_dir
    
#     def format_question(self, question, model_name):
#         """Format a question for the model."""

#         if model_name=="blip2":
#             return f"Question: {question['question']} Provide only the total number. Answer:" #Provide just the total count and the list of objects in the given format \n Format: number [objects] Answer: "
#         else:
#             return f"Question: {question['question']} Answer(total number):"

#     def clean_answer(self, answer):
#         """Clean the model output to extract just the number."""
#         # Remove any text that's not a number
#         # import re
#         # numbers = re.findall(r'\d+', answer)
#         # if numbers:
#         #     return numbers[0]  # Return the first number found
#         # return answer
#         """Extract number and reasoning from the model's answer."""
#         # Try to extract number and reasoning using regex
#         import re
#         pattern = r'(\d+)\s*\[(.*?)\]'
#         match = re.search(pattern, answer)
        
#         if match:
#             number = match.group(1)
#             objects = [obj.strip() for obj in match.group(2).split(',')]
#             return {
#                 "count": number,
#                 "reasoning": objects
#             }
#         else:
#             # Fallback if format isn't matched
#             numbers = re.findall(r'\d+', answer)
#             return {
#                 "count": numbers[0] if numbers else "0",
#                 "reasoning": []
#             }

#     def model_generation(self, model_name, model, inputs, processor):
#         """Generate answer and decode."""
#         outputs = None  # Initialize outputs to None
        
#         if model_name=="blip2":
#             outputs = model.generate(**inputs)
#             answer = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
            
#         elif model_name=="fuyu-8b":
#             outputs = model.generate(
#                 **inputs,
#                 max_new_tokens=30,  # Increased from 10 to 200
#                 pad_token_id=processor.tokenizer.eos_token_id
#             )
#             answer = processor.batch_decode(outputs[:, -30:], skip_special_tokens=True)[0]
#         else:
#             print(f"Warning: Unknown model name '{model_name}' in model_generation.")
#             answer = ""  # Return an empty string

#         return answer, outputs
    
#     def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
#         results = []
#         print(f"\nEvaluating {model_name}...")
#         print(f"Using device: {self.device}")
        
#         # Force garbage collection before starting
#         gc.collect()
#         torch.cuda.empty_cache()

#         try:
#             images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
#             total_images = len(images)
            
#             for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
#                 try:
#                     print(f"\nProcessing image {idx+1}/{total_images}: {image_data['image_id']}")
#                     image_path = Path(self.data_dir)/image_data['path']
#                     if not image_path.exists():
#                         print(f"Warning: Image not found at {image_path}")
#                         continue
                    
#                     # Load and preprocess image
#                     image = Image.open(image_path).convert("RGB")
#                     image_results = []  # Store results for current image
                    
#                     for question in image_data['questions']:
#                         try:
#                             prompt = self.format_question(question, model_name)
#                             print(f"Question: {question['question']}")
                            
#                             # Clear cache before processing each question
#                             torch.cuda.empty_cache()
                            
#                             # Process image and text
#                             inputs = processor(images=image, text=prompt, return_tensors="pt").to(self.device)
                            
#                             # Generate answer with better settings
#                             with torch.no_grad():
#                                 answer, outputs = self.model_generation(model_name, model, inputs, processor)    #call for model.generate
                                
#                             cleaned_answer = self.clean_answer(answer)
                            
#                             image_results.append({
#                                 "image_id": image_data["image_id"],
#                                 "image_type": image_data["image_type"],
#                                 "question_id": question["id"],
#                                 "question": question["question"],
#                                 "ground_truth": question["answer"],
#                                 "model_answer": cleaned_answer["count"],
#                                 "model_reasoning": cleaned_answer["reasoning"],
#                                 "raw_answer": answer,  # Keep raw answer for debugging
#                                 "property_category": question["property_category"]
#                             })
                            
#                             # Clear memory
#                             del outputs, inputs
#                             torch.cuda.empty_cache()
                            
#                         except Exception as e:
#                             print(f"Error processing question: {str(e)}")
#                             continue
                    
#                     # Add results from this image
#                     results.extend(image_results)
                    
#                     # Save intermediate results only every 2 images or if it's the last image
#                     if (idx + 1) % 2 == 0 or idx == total_images - 1:
#                         with open(f"{save_path}_checkpoint.json", 'w') as f:
#                             json.dump(results, f, indent=4)
                            
#                 except Exception as e:
#                     print(f"Error processing image {image_data['image_id']}: {str(e)}")
#                     continue
            
#             # Save final results
#             if results:
#                 with open(save_path, 'w') as f:
#                     json.dump(results, f, indent=4)
            
#         except Exception as e:
#             print(f"An error occurred during evaluation: {str(e)}")
#             if results:
#                 with open(f"{save_path}_error_state.json", 'w') as f:
#                     json.dump(results, f, indent=4)
        
#         return results

In [4]:
class BenchmarkTester:
    def __init__(self, benchmark_path="/var/scratch/ave303/OP_bench/benchmark.json", data_dir="/var/scratch/ave303/OP_bench/"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        with open(benchmark_path, 'r') as f:
            self.benchmark = json.load(f)
        self.data_dir = data_dir
    
    def format_question(self, question, model_name):
        """Format a question for the model."""
        if model_name=="blip2":
        #     return f"Question: {question['question']} Provide only the total number. Answer:"
        #     # return f"{question['question']} Your response MUST be in the following format and nothing else:\n <NUMBER> [<OBJECT1>, <OBJECT2>, <OBJECT3>, ...]"
        # else:
            return f"Question: {question['question']} Answer(total number):"

    def clean_answer(self, answer):
        """Extract number and reasoning from the model's answer."""
        # Try to extract number and reasoning using regex
        import re
        # pattern = r'(\d+)\s*\[(.*?)\]'
        # match = re.search(pattern, answer)
        
        # if match:
        #     number = match.group(1)
        #     objects = [obj.strip() for obj in match.group(2).split(',')]
        #     return {
        #         "count": number,
        #         "reasoning": objects
        #     }
        # else:
        #     # Fallback if format isn't matched
        #     numbers = re.findall(r'\d+', answer)
        #     return {
        #         "count": numbers[0] if numbers else "0",
        #         "reasoning": []
        #     }
        # Word to number mapping
        word_to_num = {
            'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5',
            'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10',
            'eleven': '11', 'twelve': '12', 'thirteen': '13', 'fourteen': '14', 'fifteen': '15',
            'sixteen': '16', 'seventeen': '17', 'eighteen': '18', 'nineteen': '19', 'twenty': '20',
            'thirty': '30', 'forty': '40', 'fifty': '50', 'sixty': '60', 'seventy': '70',
            'eighty': '80', 'ninety': '90', 'hundred': '100'
        }
        
        # Roman numeral to number mapping
        roman_to_num = {
            'i': '1', 'ii': '2', 'iii': '3', 'iv': '4', 'v': '5', 'vi': '6', 'vii': '7',
            'viii': '8', 'ix': '9', 'x': '10', 'xi': '11', 'xii': '12', 'xiii': '13',
            'xiv': '14', 'xv': '15', 'xvi': '16', 'xvii': '17', 'xviii': '18', 'xix': '19',
            'xx': '20'
        }
        
        # Convert answer to lowercase for processing
        answer_lower = answer.lower().strip()
        
        # Try to extract number and reasoning using regex first
        pattern = r'(\d+)\s*\[(.*?)\]'
        match = re.search(pattern, answer)
        
        if match:
            number = match.group(1)
            objects = [obj.strip() for obj in match.group(2).split(',')]
            return {
                "count": number,
                "reasoning": objects
            }
        else:
            # Look for regular digits first
            numbers = re.findall(r'\d+', answer)
            if numbers:
                return {
                    "count": numbers[0],
                    "reasoning": []
                }
            
            # Check for word numbers
            for word, num in word_to_num.items():
                if word in answer_lower:
                    return {
                        "count": num,
                        "reasoning": []
                    }
            
            # Check for roman numerals
            for roman, num in roman_to_num.items():
                if roman in answer_lower:
                    return {
                        "count": num,
                        "reasoning": []
                    }
            
            # If nothing found, return "0"
            return {
                "count": "0",
                "reasoning": []
            }

    def model_generation(self, model_name, model, inputs, processor):
        """Generate answer and decode with greedy decoding."""
        outputs = None  # Initialize outputs to None
        
        if model_name=="blip2":
            # Explicit greedy decoding parameters for BLIP2
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=False,          # Disable sampling for greedy decoding
                temperature=None,         # Not used in greedy decoding
                top_p=None,              # Not used in greedy decoding  
                top_k=None,              # Not used in greedy decoding
                num_beams=1,             # Single beam for greedy decoding
                early_stopping=False,    # Let it generate until max_tokens or EOS
            )
            answer = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
            
        elif model_name=="fuyu-8b":
            # Explicit greedy decoding parameters for Fuyu-8B
            outputs = model.generate(
                **inputs,
                max_new_tokens=30,
                do_sample=False,          # Disable sampling for greedy decoding
                temperature=None,         # Not used in greedy decoding
                top_p=None,              # Not used in greedy decoding  
                top_k=None,              # Not used in greedy decoding
                num_beams=1,             # Single beam for greedy decoding
                early_stopping=False,    # Let it generate until max_tokens or EOS
                pad_token_id=processor.tokenizer.eos_token_id
            )
            answer = processor.batch_decode(outputs[:, -30:], skip_special_tokens=True)[0]
        else:
            print(f"Warning: Unknown model name '{model_name}' in model_generation.")
            answer = ""  # Return an empty string

        return answer, outputs
    
    def evaluate_model(self, model_name, model, processor, save_path, start_idx=0, batch_size=5):
        results = []
        
        # Initialize tracking variables
        successful_images = []
        failed_images = []
        total_questions_processed = 0
        total_questions_failed = 0
        
        print(f"\nEvaluating {model_name}...")
        print(f"Using device: {self.device}")
        
        # Force garbage collection before starting
        gc.collect()
        torch.cuda.empty_cache()

        try:
            images = self.benchmark['benchmark']['images'][start_idx:start_idx + batch_size]
            total_images = len(images)
            
            for idx, image_data in enumerate(tqdm(images, desc="Processing images")):
                image_start_time = time.time()
                current_image_questions_failed = 0
                current_image_questions_total = 0
                
                try:
                    image_path = Path(self.data_dir)/image_data['path']
                    if not image_path.exists():
                        failed_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'error_type': 'file_not_found',
                            'error_message': f'Image not found at {image_path}'
                        })
                        continue
                    
                    # Load and preprocess image
                    image = Image.open(image_path).convert("RGB")
                    image_results = []  # Store results for current image
                    
                    for question_idx, question in enumerate(image_data['questions']):
                        current_image_questions_total += 1
                        total_questions_processed += 1
                        
                        try:
                            prompt = self.format_question(question, model_name)
                            
                            # Clear cache before processing each question
                            torch.cuda.empty_cache()
                            
                            # Process image and text
                            inputs = processor(images=image, text=prompt, return_tensors="pt").to(self.device)
                            
                            # Generate answer with greedy decoding
                            with torch.no_grad():
                                answer, outputs = self.model_generation(model_name, model, inputs, processor)
                                
                            cleaned_answer = self.clean_answer(answer)
                            
                            image_results.append({
                                "image_id": image_data["image_id"],
                                "image_type": image_data.get("image_type", "unknown"),
                                "question_id": question["id"],
                                "question": question["question"],
                                "ground_truth": question["answer"],
                                "model_answer": cleaned_answer["count"],
                                "model_reasoning": cleaned_answer["reasoning"],
                                "raw_answer": answer,  # Keep raw answer for debugging
                                "property_category": question["property_category"]
                            })
                            
                            # Clear memory
                            del outputs, inputs
                            torch.cuda.empty_cache()
                            
                        except Exception as e:
                            current_image_questions_failed += 1
                            total_questions_failed += 1
                            continue
                    
                    # Add results from this image
                    results.extend(image_results)
                    
                    # Calculate success rate for this image
                    questions_succeeded = current_image_questions_total - current_image_questions_failed
                    
                    if current_image_questions_failed == 0:
                        # All questions succeeded
                        successful_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'questions_total': current_image_questions_total,
                            'questions_succeeded': questions_succeeded,
                            'processing_time': time.time() - image_start_time
                        })
                    else:
                        # Some questions failed
                        image_success_rate = (questions_succeeded / current_image_questions_total * 100) if current_image_questions_total > 0 else 0
                        failed_images.append({
                            'image_id': image_data['image_id'],
                            'image_type': image_data.get('image_type', 'unknown'),
                            'error_type': 'partial_failure',
                            'questions_total': current_image_questions_total,
                            'questions_failed': current_image_questions_failed,
                            'questions_succeeded': questions_succeeded,
                            'success_rate': f"{image_success_rate:.1f}%"
                        })
                    
                    # Save intermediate results only every 2 images or if it's the last image
                    if (idx + 1) % 2 == 0 or idx == total_images - 1:
                        checkpoint_path = f"{save_path}_checkpoint.json"
                        with open(checkpoint_path, 'w') as f:
                            json.dump(results, f, indent=4)
                            
                except Exception as e:
                    failed_images.append({
                        'image_id': image_data['image_id'],
                        'image_type': image_data.get('image_type', 'unknown'),
                        'error_type': 'complete_failure',
                        'error_message': str(e)
                    })
                    continue
            
            # Save final results
            if results:
                with open(save_path, 'w') as f:
                    json.dump(results, f, indent=4)
            
        except Exception as e:
            if results:
                error_save_path = f"{save_path}_error_state.json"
                with open(error_save_path, 'w') as f:
                    json.dump(results, f, indent=4)
        
        # Print comprehensive summary
        self._print_evaluation_summary(
            model_name, total_images, successful_images, failed_images, 
            total_questions_processed, total_questions_failed, len(results)
        )
        
        return results
    
    def _print_evaluation_summary(self, model_name, total_images, successful_images, 
                                failed_images, total_questions_processed, total_questions_failed, total_results):
        """Print a comprehensive evaluation summary."""
        print(f"\n{'='*60}")
        print(f"EVALUATION SUMMARY FOR {model_name.upper()}")
        print(f"{'='*60}")
        
        # Image-level statistics
        num_successful = len(successful_images)
        num_failed = len(failed_images)
        
        print(f"📊 IMAGE PROCESSING SUMMARY:")
        print(f"   Total images attempted: {total_images}")
        print(f"   Successfully processed: {num_successful} ({num_successful/total_images*100:.1f}%)")
        print(f"   Failed images: {num_failed} ({num_failed/total_images*100:.1f}%)")
        
        # Question-level statistics
        questions_succeeded = total_questions_processed - total_questions_failed
        print(f"\n📝 QUESTION PROCESSING SUMMARY:")
        print(f"   Total questions attempted: {total_questions_processed}")
        print(f"   Successfully processed: {questions_succeeded} ({questions_succeeded/total_questions_processed*100:.1f}%)")
        print(f"   Failed questions: {total_questions_failed} ({total_questions_failed/total_questions_processed*100:.1f}%)")
        print(f"   Results saved: {total_results}")
        
        # Successful images details
        if successful_images:
            print(f"\n✅ SUCCESSFUL IMAGES ({len(successful_images)}):")
            for img in successful_images:
                print(f"   • {img['image_id']} (Type: {img['image_type']}, "
                      f"Questions: {img['questions_succeeded']}/{img['questions_total']}, "
                      f"Time: {img['processing_time']:.1f}s)")
        
        # Failed images details
        if failed_images:
            print(f"\n❌ FAILED/PROBLEMATIC IMAGES ({len(failed_images)}):")
            for img in failed_images:
                if img['error_type'] == 'complete_failure':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"COMPLETE FAILURE: {img.get('error_message', 'Unknown error')}")
                elif img['error_type'] == 'partial_failure':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"PARTIAL: {img['questions_failed']}/{img['questions_total']} failed "
                          f"({img['success_rate']} success)")
                elif img['error_type'] == 'file_not_found':
                    print(f"   • {img['image_id']} (Type: {img['image_type']}) - "
                          f"FILE NOT FOUND: {img['error_message']}")
        
        # Group failed images by type
        if failed_images:
            print(f"\n📈 FAILURE ANALYSIS BY IMAGE TYPE:")
            from collections import defaultdict
            failures_by_type = defaultdict(list)
            for img in failed_images:
                failures_by_type[img['image_type']].append(img)
            
            for img_type, failures in failures_by_type.items():
                print(f"   • {img_type}: {len(failures)} failed images")
                for failure in failures:
                    print(f"     - {failure['image_id']} ({failure['error_type']})")
        
        print(f"{'='*60}\n")

## Test Fuyu Model

Let's evaluate the Fuyu-8b model on our benchmark.

In [5]:
def test_fuyu():
    #from transformers import AutoModelForCausalLM, AutoTokenizer
    from transformers import FuyuProcessor, FuyuForCausalLM
    
    print("Loading Fuyu-8b model...")
    model = FuyuForCausalLM.from_pretrained(
        "/var/scratch/ave303/models/fuyu-8b",
        # load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True
    ).eval()
    processor = FuyuProcessor.from_pretrained("/var/scratch/ave303/models/fuyu-8b")

    ## fuyu-8b is very slow and average performance

    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True
        
    tester = BenchmarkTester()
    fuyu_results = tester.evaluate_model(
        "fuyu-8b",
        model, 
        processor, 
        "fuyu_8b_results.json", 
        batch_size=360
    )
    # tester.save_results("fuyu_results.json")

    if fuyu_results is not None:
        print("Initial test successful!")
    
    # Clean up
    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

## Test BLIP-2 Model

Now let's evaluate the blip2 model.

In [6]:
def test_blip2():
    from transformers import Blip2Processor, Blip2ForConditionalGeneration
    
    print("Loading BLIP-2 model...")
    model = Blip2ForConditionalGeneration.from_pretrained(
        "/var/scratch/ave303/models/blip2flan-t5-xxl",
        # load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="auto",
        # temperature=0.8,
        low_cpu_mem_usage=True
    ).eval()
    processor = Blip2Processor.from_pretrained("/var/scratch/ave303/models/blip2flan-t5-xxl")

    ## opt-2.7b average performance, better instruction following 
        # Format - Answer(total number):
    ## opt-6.7b(8bit) better performance with atleast answering, not well-instruction tuned, but provides number for answers
        # Format - Answer(total number):
    ## flan-t5-xl does fine but needs a lot of post processing, does not follow instructions to clearly
        # Format - Answer(provide total number):
    ## flan-t5-xxl(8bit) decent performance, better with instruction I think, slight postprocessing needed
        # Format - Answer:
    
    # Optional: Enable memory efficient attention
    if hasattr(model.config, 'use_memory_efficient_attention'):
        model.config.use_memory_efficient_attention = True
    
    tester = BenchmarkTester()
    blip2_results = tester.evaluate_model(
        "blip2",
        model, 
        processor, 
        "blip2-flan-t5-xxl_results.json", 
        batch_size=360
    )
    # tester.save_results("blip2_results.json")

    if blip2_results is not None:
        print("Initial test successful!")
    
    # Clean up
    del model, processor
    torch.cuda.empty_cache()
    gc.collect()

## Run Evaluation

Now we can run our evaluation. Let's start with the Fuyu model:

In [7]:
# test_fuyu()

And then the BLIP-2 model:

In [8]:
test_blip2()

/var/scratch/ave303/anaconda3/envs/op_bench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BLIP-2 model...


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Loading checkpoint shards:  10%|█         | 1/10 [00:05<00:45,  5.07s/it]

Loading checkpoint shards:  20%|██        | 2/10 [00:09<00:39,  4.89s/it]

...

Processing images:  99%|█████████▉| 358/360 [03:25<00:00,  2.08it/s]

Processing images: 100%|█████████▉| 359/360 [03:25<00:00,  1.97it/s]

Processing images: 100%|██████████| 360/360 [03:26<00:00,  1.96it/s]

Processing images: 100%|██████████| 360/360 [03:26<00:00,  1.75it/s]


EVALUATION SUMMARY FOR BLIP2
📊 IMAGE PROCESSING SUMMARY:
   Total images attempted: 360
   Successfully processed: 360 (100.0%)
   Failed images: 0 (0.0%)

📝 QUESTION PROCESSING SUMMARY:
   Total questions attempted: 1080
   Successfully processed: 1080 (100.0%)
   Failed questions: 0 (0.0%)
   Results saved: 1080

✅ SUCCESSFUL IMAGES (360):
   • image01 (Type: REAL, Questions: 3/3, Time: 1.7s)
   • image02 (Type: REAL, Questions: 3/3, Time: 0.6s)
   • image03 (Type: REAL, Questions: 3/3, Time: 0.5s)
   • image04 (Type: REAL, Questions: 3/3, Time: 0.5s)
   • image05 (Type: REAL, Questions: 3/3, Time: 0.5s)
   • image06 (Type: REAL, Questions: 3/3, Time: 0.5s)
   • image07 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • image08 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • image09 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • image10 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • image11 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • image12 (Type: REAL, Questions: 3/3, Time: 0.4s)
   • im